# Import packages

In [13]:
#conda update scikit-learn
#conda install -c conda-forge sklearn-pandas
import pandas as pd 
import os
import datetime as dt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn_pandas import DataFrameMapper

import hashlib as h


## Interesting definition 

- k-anonymous : A table is k-anonymous if every record in the table is indistinguishable from at least k-1 other records, with respect to every set of quasi-identifiers
- l-diverse :  An equivalence class is l-diverse if it contains at least l distinct values for the sensitive attributes. A table is l-diverse if every equivalence class is l-diverse

# Directory choice

In [162]:
data = pd.read_csv("../resources/dataset_HW3.csv")

# Visualazing data

In [163]:
#pd.reset_option('all')
data

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,female,12/1950,26904,High School,Retired,2,married,Africa,2,0.0,Own house,Alzheimer's disease
1,Lillian Cole,female,4/1940,78088,PhD/md,Retired,1,married,Africa,1,0.0,Rent flat,multiple sclerosis
2,Lillian Simpson,female,12/1966,51502,Bachelor,Employed,1,married,Asia,0,0.1,Rent flat,heart disease
3,Avery Richards,female,4/1943,54080,Bachelor,Retired,1,married,Africa,0,0.0,Rent room,endometriosis
4,Sophia Alvarado,female,10/1947,68785,High School,Retired,1,married,Europe,0,0.0,Rent flat,gastritis
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Connor Pena,male,11/1997,71300,High School,Student,0,divorced,Europe,0,1.1,Rent house,gastritis
1996,Caleb Bowman,male,11/1999,49030,High School,Student,0,single,North America,0,0.1,Rent house,gastritis
1997,Ella Barber,female,6/1948,92883,Masters,Retired,5,married,Africa,1,0.0,Own house,skin cancer
1998,Gavin Wilson,male,4/1976,42200,High School,Employed,1,married,Asia,2,0.0,Own flat,diabetes


#### For sociologists, to study the impact of stress and high-pressure environments on one’s health. They are looking for factors in people’s lives that could correlate to certain diseases or conditions. : 

They don't need to know any information about the identity of individuals. However, gender, dob and localisaiton  is really important information : 

- The id ( by name could be dropped (BE CARREFUL BECAUSE THIS INFORMATION COULD BE LINKED WITH THE SECOND PART OF THE EXERCICE)) 
- The gender could be replaced by 0 or 1 ( mais en vrai ca change pas grand chose)
- The dob : we don't need the day and the month of birth ==> Ne pourrait-on pas modifier grâce à une gaussienne de moyenne 0 sd 1 les dates de naissance ? On gagne en anonymisation, on ne perd pas grand chose 
- Zip code, we do not need the entire zipcode, we can keep only the first to digit.

==> Etant donnél'étude demandé, le risque n'est pas les identifiant mais les pseusos identifiants.

#### For the US department of Health and Human services chairman, to decide where to build new hospitals, and which departments (radiology, neurology, pulmonology, oncology...) to have in these, as the US federal government has just allocated money to build *5 new hospitals across the country* : 

A priori, je ne vois pas pourquoi on aurait besoin d'information sur les individus, ce qui nous importe est uniquement les maladies, la localicaiton, et pourquoi pas l'age des gens.

- Gender ne sert plus à rien
- l'année de naissance suffit 
- les deux premiers digit du zip code sont suffisant (étant donné que l'on va faire que 5 hopitauyx across the country)
- L'éducation on s'en branle
- Emploie on s'en fou
- Nombre d'enfant on s'en fou 
- Marital status on s'en branle
- ancestry on s'en fou 
- Nombre de véhicule ( est ce que ca peut être intéressant étant donné que c'est des hopitaux? ==> Ambulance tout ca)
- communte-time on s'en branle 
- accommodation on s'en fou 
- Maladie, pourrait on déjà regrouper les différentes maladies selon le service utilise à soigner les tiches ? On s'en branle de savoir si c'est un cancer du foie ou de la peau, ce qui est intéressant c'est qu'il faut un service d'oncologie



## Mofification de la base de données

In [8]:
#print(data["dob"])
new_dob = [0]*(len(data["dob"]))

for i in range(len(data["dob"])):

    # Modify DOB 
    s = np.random.normal(0, 2, 1)
    data.at[i,"dob"] = round(int(data["dob"][i][-4:]) +s[0])

    # Modify ZipCode (3 first digit needed pour the city, 2 or 3 for the state, 1 can be different state
    # https://simple.wikipedia.org/wiki/List_of_ZIP_Code_prefixes#:~:text=2%20%3D%20District%20of%20Columbia%20(DC,)%2C%20West%20Virginia%20(WV))

    data.at[i,"zipcode"] = int((str(data["zipcode"][i])[:3]))


## Variables catégorielles 

In [168]:
def change_education(data, less, high, bach, master, phd):

    mapping_education = {
        'Less than High School':less,
        'High School':high,
        'Bachelor':bach,
        'Masters':master,
        'PhD/md':phd,
    }

    data["education"] = data.education.map(mapping_education)


def change_employment(data, employed, retired, student, unemployed):
    mapping_employment = {
        'Employed' : employed,
        'Retired' : retired,
        'Student' : student,
        'Unemployed' : unemployed
    }

    data["employment"] = data.employment.map(mapping_employment)


def change_children(data, zero, un, deux, trois, quatre , cinq, six, sept, huit):
    mapping_children = {
        0 : zero,
        1 : un,
        2 : deux,
        3 : trois,
        4 : quatre,
        5 : cinq,
        6 : six,
        7 : sept,
        8 : huit
    }

    data["children"] = data.children.map(mapping_children)

def change_marital_status(data, divorced, married, single, widowed):
    mapping_marital_status = {
        "divorced" : divorced,
        "married" : married,
        "single" : single,
        "widowed" : widowed,

    }

    data["marital_status"] = data.marital_status.map(mapping_marital_status)


def change_ancestry(data, Africa, Asia, Europe, North_America, Oceania, South_America):
    mapping_ancestry = {
        "Africa" : Africa,
        "Asia" : Asia,
        "Europe" : Europe,
        "North America" : North_America,
        "Oceania" : Oceania,
        "South America" : South_America

    }

    data["ancestry"] = data.ancestry.map(mapping_ancestry)


def change_number_vehicles(data, zero, un, deux, trois):
    mapping_nmbr_vehicule = {
        0 : zero,
        1 : un,
        2 : deux,
        3 : trois,

    }

    data["number_vehicles"] = data.number_vehicles.map(mapping_nmbr_vehicule)


def change_accommodation(data, Own_flat, Own_house, Rent_flat, Rent_house, Rent_room):
    mapping_accommodation = {
        "Own flat" : Own_flat,
        "Own house" : Own_house,
        "Rent flat" : Rent_flat,
        "Rent house" : Rent_house,
        "Rent room" : Rent_room,
        
    }

    data["accommodation"] = data.accommodation.map(mapping_accommodation)

# for the sake of the first use case we gather all specific diseases under more general health department 

def change_disease(data, gastritis, heart_disease, hypertension, diabetes, skin_cancer, prostate_cancer, breast_cancer, schizophrenia, Alzheimer, multiple_sclerosis, endometriosis, kidney_disease, HIV_AIDS):
    mapping_disease = {
        'gastritis': gastritis ,
        'heart disease': heart_disease,
        "hypertension": hypertension,
        "diabetes": diabetes,
        "skin cancer": skin_cancer,
        "prostate cancer":  prostate_cancer,
        "breast cancer": breast_cancer, 
        "schizophrenia": schizophrenia,
        "Alzheimer's disease": Alzheimer, 
        "multiple sclerosis": multiple_sclerosis,
        "endometriosis": endometriosis, 
        "kidney disease": kidney_disease,
        "HIV/AIDS": HIV_AIDS 
    }

    data['disease'] = data.disease.map(mapping_disease)


In [165]:
# 'low' under highschool, 'high' elsewhere 
change_education(data, less= 'low', high='low', bach='high', master='high', phd='high')

# active = {employed, student} unactive = {retired, unemployed}
change_employment(data, employed= 'active', retired='unactive', student='active', unemployed='unactive')

# QUE FAIRE AVEC LES CHILDREN ? 

# married = married, unmarried = others
change_marital_status(data, divorced='unmarried', married='married', single='unmarried', widowed='unmarried')

# native : north america, foreigners : others BUT to much 'other' compare to native so might affect negatively l-diversity. Instead, we merge north/south america 
change_ancestry(data, Africa='Africa', Asia='Asia', Europe='Europe', North_America='America', Oceania='Oceania', South_America='America')

# 1 if got vehicule, 0 else 
change_number_vehicles(data, zero='none', un='one', deux='more than one', trois='more than one')

# owner = owner, renter = renter regardless of the type of the accomodaations (2/3 owner, maybe reset these proportion ? )
change_accommodation(data, Own_flat='owner', Own_house='owner', Rent_flat= 'renter', Rent_house='renter', Rent_room='renter')

# gathered under medical departments
change_disease(data, gastritis='gastro-enterolgy', heart_disease='cardiovascular', hypertension='cardiovascular', diabetes='endocrynology', 
    skin_cancer='oncology', prostate_cancer='oncology', breast_cancer='oncology', 
    schizophrenia='psychiatry', Alzheimer='neurology', multiple_sclerosis='neurology', endometriosis='gynecology', kidney_disease='nephrology', HIV_AIDS='interal mdeicine')

# 

In [169]:

data["education"].value_counts()
data["employment"].value_counts()
data["marital_status"].value_counts()
data["ancestry"].value_counts()
data["number_vehicles"].value_counts()
data["accommodation"].value_counts()
data["disease"].value_counts()


cardiovascular      459
oncology            295
neurology           280
gastro-enterolgy    255
interal mdeicine    159
endocrynology       149
gynecology          143
nephrology          131
psychiatry          129
Name: disease, dtype: int64

## Change ID by hashing

In [ ]:
d = {}

for id in data['id']:
    id_salted = str(id.replace(' ', '')+'arthurgeeraerd11751600')
    new_id = h.sha3_256(id_salted.encode()).hexdigest()
    d[id] = new_id

data_socio = data.replace(d)
d.clear()

# second dataset for the purpose of the 2nd use case (using salt : 'simonadrienkinartlegendre)

for id in data['id']:
    id_salted = str(id.replace(' ', '')+'simonadrienkinartlegendre')
    new_id = h.sha3_256(id_salted.encode()).hexdigest()
    d[id] = new_id

data_hospi = data.replace(d)
d.clear()

## Modification supplémentaire

In [9]:
# We drop ID
data.drop(columns=["id"], inplace=True)

# Binariser certaines variables ( gender a déjà été fait)
"""data = pd.get_dummies(data, columns=["disease", "education", "employment", "marital_status", "ancestry", "accommodation"], 
                        prefix=["disease", "education", "employment", "marital_status", "ancestry", "accommodation"])"""

# Autre méthode
encoders = [(["gender"], LabelEncoder()), (["disease"], LabelEncoder()), (["education"], LabelEncoder()),(["employment"], LabelEncoder()), 
            (["marital_status"], LabelEncoder()),(["ancestry"], LabelEncoder()),(["accommodation"], LabelEncoder())]

mapper = DataFrameMapper(encoders, df_out=True)
new_cols = mapper.fit_transform(data.copy())

data = pd.concat([data.drop(columns=["gender","disease", "education", "employment", "marital_status", "ancestry", "accommodation"]),
        new_cols], axis="columns")


C:\Users\arthu\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
data.nunique()

dob                 79
zipcode            806
children             9
number_vehicles      4
commute_time        12
gender               2
disease             13
education            5
employment           4
marital_status       4
ancestry             6
accommodation        5
dtype: int64

In [12]:
# On regroupe les variables categorielles et continues entre elles

categorical = []
continuous = []

for c in list(data):
    col = data[c]
    nunique = col.nunique()
    if nunique < 20:
        categorical.append(c)
    else:
        continuous.append(c)


print(categorical)
print(continuous)

['children', 'number_vehicles', 'commute_time', 'gender', 'disease', 'education', 'employment', 'marital_status', 'ancestry', 'accommodation']
['dob', 'zipcode']
